In [ ]:
# default_exp web

# Web
>This module contain functions to process the model outputs for web applications.

In [ ]:
# hide
from nbdev.showdoc import show_doc
from nbdev.export import notebook2script

In [ ]:
# export
import PIL
import matplotlib
import numpy as np

In [ ]:
# export
def array2png(data, cmap='RdYlGn_r'):
    "Convert uint8 2d-array 'data' to png using colormap 'cmap'"
    mask = data == 0
    unique = sorted(np.unique(data[data>0]).reshape(-1).tolist())
    cmap = matplotlib.cm.get_cmap(cmap)
    colors = (cmap(np.linspace(0,1,unique[-1]-unique[0]+1))*255).astype(np.uint8)
    data[data>0] -= unique[0]
    data = data.astype(np.uint8)
    data = colors[data]
    data[mask, -1] = 0
    return PIL.Image.fromarray(data)

In [ ]:
# hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_geo.ipynb.
Converted 02_data.ipynb.
Converted 03_models.ipynb.
Converted 04_predict.ipynb.
Converted 04b_nrt.ipynb.
Converted 04c_historical.ipynb.
Converted 05_train.ipynb.
Converted 06_cli.ipynb.
Converted 07_web.ipynb.
Converted index.ipynb.
Converted tutorial.australia2020.ipynb.
Converted tutorial.australia2020_100m.ipynb.
